### SQL in 10 Minutes
#### Lesson VII
`UNION`, `INSERT`, `UPDATE`, `DELETE`

In [1]:
import sqlite3
import pandas as pd

In [2]:
db = 'tysql.sqlite'
conn = sqlite3.connect(db)

In [3]:
tables = pd.read_sql("""SELECT * FROM sqlite_master WHERE type = 'table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,Customers,Customers,2,CREATE TABLE Customers\n(\n cust_id char...
1,table,OrderItems,OrderItems,4,CREATE TABLE OrderItems\n(\n order_num int ...
2,table,Orders,Orders,7,CREATE TABLE Orders\n(\n order_num int ...
3,table,Products,Products,10,CREATE TABLE Products\n(\n prod_id char(10...
4,table,Vendors,Vendors,12,CREATE TABLE Vendors\n(\n vend_id char(1...


In [4]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_state in ('IL', 'IN', 'MI');
        """

In [5]:
ex1 = pd.read_sql(sql, conn)
ex1

,cust_name,cust_contact,cust_email
0,Village Toys,John Smith,sales@villagetoys.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,The Toy Store,Kim Howard,None


In [6]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_name = 'Fun4All';
        """

In [7]:
ex2 = pd.read_sql(sql, conn)
ex2

,cust_name,cust_contact,cust_email
0,Fun4All,Jim Jones,jjones@fun4all.com
1,Fun4All,Denise L. Stephens,dstephens@fun4all.com


Now combine the above two query.

In [8]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_state in ('IL', 'IN', 'MI')
        union
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_name = 'Fun4All';
        """

In [9]:
ex3 = pd.read_sql(sql, conn)
ex3

,cust_name,cust_contact,cust_email
0,Fun4All,Denise L. Stephens,dstephens@fun4all.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,The Toy Store,Kim Howard,None
3,Village Toys,John Smith,sales@villagetoys.com


This equals multiple `where`

In [10]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_state in ('IL', 'IN', 'MI') 
        or cust_name = 'Fun4All';
        """

In [11]:
ex4 = pd.read_sql(sql, conn)
ex4

,cust_name,cust_contact,cust_email
0,Village Toys,John Smith,sales@villagetoys.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,Fun4All,Denise L. Stephens,dstephens@fun4all.com
3,The Toy Store,Kim Howard,None


To avoid automatic cancellation, use `union all`

In [12]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_state in ('IL', 'IN', 'MI')
        union all
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_name = 'Fun4All';
        """

In [13]:
ex5 = pd.read_sql(sql, conn)
ex5

,cust_name,cust_contact,cust_email
0,Village Toys,John Smith,sales@villagetoys.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,The Toy Store,Kim Howard,None
3,Fun4All,Jim Jones,jjones@fun4all.com
4,Fun4All,Denise L. Stephens,dstephens@fun4all.com


`order by`

In [14]:
sql = """
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_state in ('IL', 'IN', 'MI')
        union 
        select cust_name, cust_contact, cust_email
        from Customers
        where cust_name = 'Fun4All'
        order by cust_name, cust_contact;
        """

In [15]:
ex6 = pd.read_sql(sql, conn)
ex6

,cust_name,cust_contact,cust_email
0,Fun4All,Denise L. Stephens,dstephens@fun4all.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,The Toy Store,Kim Howard,None
3,Village Toys,John Smith,sales@villagetoys.com


`Insert` is another type of sql language apart from `Select`

In [18]:
sql = """
        insert into Customers(cust_id, cust_name, cust_address, cust_city, cust_state, cust_zip, cust_country, cust_contact, cust_email)
        values('1000000006', 'Toy Land', '123 Any Street', 'New York', 'NY', '11111', 'USA', Null, Null);
        """

This is an example, and if exerted there will be `unique constrained failed` error reported.

But what's important is to give exactly the name of the column you want to insert new data in.

Here, we use another way to present the work:

In [17]:
bck1 = sqlite3.connect(db)

In [19]:
cur = bck1.cursor()
cur.execute(sql)

In [20]:
bck1.commit()

In [21]:
ex7 = pd.read_sql("select * from Customers where cust_id = 1000000006;", bck1)
ex7

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000006,Toy Land,123 Any Street,New York,NY,11111,USA,None,None


`insert select`

In [22]:
sql = """
        insert into Customers(cust_id, cust_name, cust_address, cust_city, cust_state, cust_zip, cust_country, cust_contact, cust_email)
        select cust_id, cust_name, cust_address, cust_city, cust_state, cust_zip, cust_country, cust_contact, cust_email
        from Custnew;
        """

`select into`

In [23]:
sql = """
        create table new_copy as
        select *
        from Customers;
        """

In [24]:
bck2 = sqlite3.connect(db)
cur = bck2.cursor()
cur.execute(sql)
bck2.commit()

In [25]:
ex8 = pd.read_sql("select * from new_copy;", bck2)
ex8

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None
5,1000000006,Toy Land,123 Any Street,New York,NY,11111,USA,None,None


See that we create a brandnew table above.

In [26]:
tables = pd.read_sql("""SELECT * FROM sqlite_master WHERE type = 'table';""", bck2)
tables

,type,name,tbl_name,rootpage,sql
0,table,Customers,Customers,2,CREATE TABLE Customers\n(\n cust_id char...
1,table,OrderItems,OrderItems,4,CREATE TABLE OrderItems\n(\n order_num int ...
2,table,Orders,Orders,7,CREATE TABLE Orders\n(\n order_num int ...
3,table,Products,Products,10,CREATE TABLE Products\n(\n prod_id char(10...
4,table,Vendors,Vendors,12,CREATE TABLE Vendors\n(\n vend_id char(1...
5,table,new_copy,new_copy,15,"CREATE TABLE new_copy(\n cust_id TEXT,\n cus..."


`UPDATE`

First we see the missing information from the wanted row.

In [31]:
sql = """
        select * 
        from Customers
        where cust_id = '1000000005';
        """

In [32]:
ex9 = pd.read_sql(sql, conn)
ex9

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


In [33]:
sql = """
        update Customers
        set cust_email = 'kim@thetoystore.com'
        where cust_id = '1000000005';
        """

In [34]:
bck3 = sqlite3.connect(db)
cur = bck3.cursor()
cur.execute(sql)
bck3.commit()

Check if it works:

In [35]:
ex10 = pd.read_sql("select * from Customers where cust_id = 1000000005;", bc3)
ex10

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,kim@thetoystore.com


`UPDATE` multiple rows

In [36]:
sql = """
        update Customers
        set cust_contact = 'Sam Roberts',
             cust_email = 'sam@toyland.com'
        where cust_id = '1000000006';
        """

In [37]:
bck4 = sqlite3.connect(db)
cur = bck4.cursor()
cur.execute(sql)
bck4.commit()

In [38]:
ex11 = pd.read_sql("select * from Customers where cust_id = 1000000006;", bck4)
ex11

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000006,Toy Land,123 Any Street,New York,NY,11111,USA,Sam Roberts,sam@toyland.com


In [39]:
sql = """
        update Customers
        set cust_email = Null
        where cust_id = '1000000005';
        """

In [40]:
bck5 = sqlite3.connect(db)
cur = bck5.cursor()
cur.execute(sql)
bck5.commit()

In [41]:
ex11 = pd.read_sql("select * from Customers where cust_id = 1000000005;", bck5)
ex11

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


`DELETE`

In [42]:
sql = """
        delete 
        from Customers
        where cust_id = '1000000006';
        """

In [44]:
cur = conn.cursor()
cur.execute(sql)
conn.commit()

In [45]:
ex12 = pd.read_sql("select * from Customers", conn)
ex12

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None
